# **(一)比赛介绍**
   > 最早的聊天机器人（ChatBot）ELIZA诞生于1966年，由麻省理工学院（MIT）的约瑟夫·魏泽鲍姆（Joseph Weizenbaum）开发，用于在临床治疗中模仿心理医生。也正是通过 ELIZA 的实验引发了人们对“智能机器”的思考，开启了智能聊天机器人时代。ELIZA 诞生于那个计算机时间被当作宝贵资源的年代，仅仅用于模拟笔谈。
##    *难点：环境的搭建*
   我先使用了[wechaty_js](https://github.com/wechaty/wechaty),按照官方文档中的readme.md很容易搭建起一个服务，根据克隆下来的文件大概看了下源码中提供了哪些接口，根据example，自己可以容易的实现哪些功能。（本人虽然做项目时候写过一点js的东西，但实现一个bot实在不想用js来解决，就使用深度学习模型来说，js还是不太好用，所以转向[wechaty_python](https://github.com/wechaty/python-wechaty)）.
   先使用一台PC作为服务器（win或者linux,都试过，可行）搭建好wechaty的[docker](https://github.com/wechaty/wechaty/issues/66),扫码登录wechat（在此过程中建议使用小号微信，后面测试功能的时候避免因为消息打扰他人）。

# **(二)个人思路**

  随着时代的发展，群聊bot以微软小冰为代表，逐渐进入互联网er的视野。以使用范围最广的wechat为道路，以baidu家的wechaty为载具，再载以各式model，理论上可以生成各式所需要的聊天bot。
  
  比赛提供了100h的算力卡，理论上可以在AI STUDIO平台训练出符合期望的model（本人比较懒，就直接调用了社区曾提供的[对联模型](https://aistudio.baidu.com/aistudio/projectdetail/690991?channelType=0&channel=0)了）
  
  本团队制作了有如下功能的bot，测试过程中发现bot对所有的文字都进行回复效果超级不友好（有时上传任务后人就离开了，打开微信发现傻逼bot在所有群里捣乱，害我差点被踢），所以加固定的启动词才会回复：

![](https://ai-studio-static-online.cdn.bcebos.com/74885a3312164b4c9c4a4e846ba455e92c86b9f5baac45408fd1fa2ef7706e55)
![](https://ai-studio-static-online.cdn.bcebos.com/b273571ee86e482786839db99f7a4802ad413dbac8364a07b5102db3146d8e69)
![](https://ai-studio-static-online.cdn.bcebos.com/36b75f28d27c4fa6bd4d078b0fae36531da0006b051d44ad87cf2b2bcfeec693)






  ## [B站演示链接](https://www.bilibili.com/video/BV1xB4y1c7GE)

# **(三)具体方案**

   本项目以baidu提供的[对联生成模型](https://aistudio.baidu.com/aistudio/projectdetail/690991?channelType=0&channel=0)为例，下面介绍一下这种模型：
   
   [ernie_gen_couplet](https://www.paddlepaddle.org.cn/hubdetail?name=ernie_gen_couplet&en_category=TextGeneration):ERNIE-GEN 是面向生成任务的预训练-微调框架，首次在预训练阶段加入span-by-span 生成任务，让模型每次能够生成一个语义完整的片段。在预训练和微调中通过填充式生成机制和噪声感知机制来缓解曝光偏差问题。此外, ERNIE-GEN 采样多片段-多粒度目标文本采样策略, 增强源文本和目标文本的关联性，加强了编码器和解码器的交互。ernie_gen_couplet采用开源对联数据集进行微调，可用于生成下联。下图是模型结构：
   
   ![](https://ai-studio-static-online.cdn.bcebos.com/2cd82577bed84c4f83d77523e09929e3a4801ac0967c40229a21087cbe7bea72)
   [ernie_tiny_couplet](https://www.paddlepaddle.org.cn/hubdetail?name=ernie_tiny_couplet&en_category=TextGeneration):ernie_tiny_couplet是一个对联生成模型，它由ernie_tiny预训练模型经PaddleHub TextGenerationTask微调而来，仅支持预测，如需进一步微调请参考PaddleHub text_generation demo。预训练模型转换成预测module的转换方式请参考Fine-tune保存的模型如何转化为一个PaddleHub Module。下图是模型结构：
   
   ![](https://ai-studio-static-online.cdn.bcebos.com/f756d77d8c9744319558e59017ac2bb0b93e3d67f8764c5dbc414f786457116f)
   
   该模型可以通过API的方式一键生成模型，参考：
   
    

      
      
   

In [ ]:
import paddlehub as hub

module = hub.Module(name="ernie_gen_couplet")

test_texts = ["人增福寿年增岁", "风吹云乱天垂泪"]
results = module.generate(texts=test_texts, use_gpu=False, beam_width=5)
for result in results:
    print(result)

   本项目详细代码请参考work/run.py文件，大致流程为：
   监听消息->启动关键词->启动模型进行对联生成->进行回复
   需要对消息进行筛选，只对被艾特的时候进行回复：
   
![](https://ai-studio-static-online.cdn.bcebos.com/583584482dfb4a4b9fafeeb60806e0da132536508e0047fba8fd137429cc396a)

不然会这样（这俩个都是我的机器人）：

![](https://ai-studio-static-online.cdn.bcebos.com/5e84412de9c04b2ea06742eebe59e0f8ab791defd49c46009e0e7895caad7fcf)

*运行*

   1 按照[教程](https://github.com/wechaty/python-wechaty-getting-started)搭建docker wechaty服务提供token,个人请更换run.sh中的token值
   
   2 具体代码位于run.py中，在run.py中设置昵称，指示机器人只在群聊中被艾特的情况下回复，不回复未被艾特的消息。contact_id为指示ID，可设置后不回复指定人消息，逻辑可自己更改

# **(四)总结**

使用wechaty可以按自己需要制作个定时给自己发送通知的个人助手，后续看情况完善。

还是想制作一个可以正常进行多轮对话的聊天bot，目前没看到适用的效果好的开放model，后续会跟进这方面发展。


# **(五)建议**

AI STUDIO平台就邮费力度来说挺值的，方便我们贫农玩model,提供的预训练模型也够自己玩的，目前还没遇到bug,可能自己玩的不够深入吧。

# **(六)参考资料**
1. https://github.com/wechaty/wechaty
2. https://github.com/wechaty/python-wechaty
3. https://github.com/wechaty/wechaty/issues/66
4. https://aistudio.baidu.com/aistudio/projectdetail/690991?channelType=0&channel=0
5. https://github.com/wechaty/python-wechaty-getting-started